# Auto-analyze Zoom chats

Find who was most active on Zoom

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
import re

In [ ]:
folder_name = '../../data/attendance/chats'

In [ ]:
# Create a list of folders to analyze

token = 'Neuroscience'

file_list = os.listdir(folder_name)
file_list = [s for s in file_list if token in s]
# file_list

In [ ]:
# Populate the dataframe

df = pd.DataFrame(columns=['date','name'])

for dirname in file_list:
    fname = folder_name + '/' + dirname + '/meeting_saved_chat.txt'
    date = dirname.split(' ')[0]
    #print(date)
    with open(fname, 'r', encoding='Utf8') as f:
        done = set()
        for l in f: # Every row
            #print(l.split(' : ')[0].split('From ')[1])            
            name = re.findall(r"From .+ : ", l)
            if name:               
                name = name[0][5:-3].strip()
                if 'Privately' in name:
                    name = name.split(' to ')[0].strip()            
                if name not in done:
                    #print(name)
                    done |= {name}
                    df = df.append({'date':date, 'name':name}, ignore_index=True)

In [ ]:
print('Total different names:', len(set(df.name.tolist())))

In [ ]:
df2 = (df
 .groupby('name')
 .agg({'date': 'count'})
 .reset_index()
 .rename({'date':'number'}, axis=1)
 .sort_values(by='number', ascending=False)
 .groupby('number')
 .agg({'name': lambda x: ', '.join(x)}) 
 .sort_values('number', ascending=False)
 .reset_index()
)

df2

for i in range(df2.shape[0]):
    print(df2.loc[i, 'number'], ':', df2.loc[i, 'name'])